# Image Autoencoders experiments

In [2]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

%matplotlib inline

Some code comes from github [SherlockLiao](https://github.com/SherlockLiao/pytorch-beginner/tree/master/08-AutoEncoder)